---

# **IT Training Assignment Class 5**

---

### **Import Required Libraries**

In [28]:
import os
import glob
import shutil
import pandas as pd

import yfinance as yf
import sqlite3
import time

import requests
from bs4 import BeautifulSoup

### **Task 01: Automated File Management & Data Export**

In [2]:
# Create and ensure that backup_folder exists

if not os.path.exists("backup_folder"):
  os.makedirs("backup_folder")

In [3]:
# Move all CSV files from "csv_files" to "backup_folder"

csv_files = glob.glob("csv_files/*.csv")
for file in csv_files:
  shutil.move(file, "backup_folder/")
  print(f"Moved file: {file}")

In [4]:
# Automating Export

def export_data(df, filename, file_format):
  if file_format == "csv":
    df.to_csv(filename, index=False)
    print(f"Data exported to {filename} in CSV format.")
  elif file_format == "json":
    df.to_json(filename, orient="records")
    print(f"Data exported to {filename} in JSON format.")
  else:
    print("Unsupported format.")

In [5]:
# Example usage: Creating a sample dataframe
data = {'Name': ['Alice', 'Bob', 'Charlie'],
        'Age': [25, 30, 35],
        'City': ['New York', 'Los Angeles', 'Chicago']}

df = pd.DataFrame(data)

# Export to CSV
export_data(df, "output.csv", "csv")

# Export to JSON
export_data(df, "output.json", "json")

Data exported to output.csv in CSV format.
Data exported to output.json in JSON format.


### **Task 02: Real-Time Stock Market Data Collection and Analysis Using  Python and SQLite**

In [15]:
# Setup the stocks database

db_name = "stocks.db"
conn = sqlite3.connect(db_name)
cursor = conn.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS stock_data (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                symbol TEXT,
                timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
                open REAL,
                high REAL,
                low REAL,
                close REAL,
                volume INTEGER)''')
conn.commit()

In [16]:
# Implement function to fetch stock data from Yahoo Finance

def fetch_stock_data(symbol):
  try:
    stock = yf.Ticker(symbol)
    data = stock.history(period="1d", interval="1m")

    if data.empty:
      print(f"No data found for {symbol}. Skipping...")
      return None  # Return None if no data is available

    latest = data.iloc[-1]  # Get the most recent price data
    return {
        "symbol": symbol,
        "open": latest["Open"],
        "high": latest["High"],
        "low": latest["Low"],
        "close": latest["Close"],
        "volume": latest["Volume"]
    }
  except Exception as e:
    print(f"Error fetching data for {symbol}: {e}")
    return None

In [17]:
# Implement function to store fetched data in sqlite

def store_data(symbol):
  stock_data = fetch_stock_data(symbol)
  if stock_data:  # Only store if data is available
    cursor.execute('''INSERT INTO stock_data (symbol, open, high, low, close, volume)
                      VALUES (?, ?, ?, ?, ?, ?)''',
                    (stock_data["symbol"], stock_data["open"],
                    stock_data["high"], stock_data["low"],
                    stock_data["close"], stock_data["volume"]))
    conn.commit()
    print(f"Stored data for {symbol}")

In [18]:
# Implement function to analyze stored data

def analyze_stock(symbol):
  df = pd.read_sql_query("SELECT * FROM stock_data WHERE symbol=? ORDER BY timestamp DESC LIMIT 100",
                          conn, params=(symbol,))
  print(df)

In [27]:
# Example Usage
symbol = "GOOGL"  # Change to any stock symbol (e.g., GOOGL for Google)

for _ in range(5):  # Fetch data 5 times with intervals
    store_data(symbol)
    time.sleep(0.1)  # Wait

analyze_stock(symbol)

# Close database connection
conn.close()

Stored data for GOOGL
Stored data for GOOGL
Stored data for GOOGL
Stored data for GOOGL
Stored data for GOOGL
   id symbol            timestamp   open   high    low  close   volume
0   1  GOOGL  2025-02-20 19:09:25  142.5  145.2  141.8  144.0  3200000
1   2  GOOGL  2025-02-20 19:09:25  144.0  146.0  143.5  145.5  3100000
2   3  GOOGL  2025-02-20 19:09:25  145.5  147.3  144.8  146.9  3300000
3   4  GOOGL  2025-02-20 19:09:25  146.9  148.0  146.2  147.5  2900000
4   5  GOOGL  2025-02-20 19:09:25  147.5  149.1  147.0  148.6  3100000


### **Task 03: Web Scraping For Bookstore Data**

In [29]:
# Set parameters for web scraping

URL = "http://books.toscrape.com/"
HEADERS = {"User-Agent": "Mozilla/5.0"}

In [30]:
# Implement function to fetch/scrape books given url

def get_books(url):
  response = requests.get(url, headers=HEADERS)
  if response.status_code != 200:
    print("Failed to retrieve the website. Check the URL.")
    return []

  soup = BeautifulSoup(response.text, "html.parser")
  books = soup.find_all("article", class_="product_pod")
  book_list = []

  for book in books:
    title = book.h3.a["title"]
    price = book.find("p", class_="price_color").text
    stock = book.find("p", class_="instock availability").text.strip()

    book_list.append({"Title": title, "Price": price, "Availability": stock})

  return book_list

In [31]:
# Example Usage - Scrape books from website
books_data = get_books(URL)

# Save to CSV
df = pd.DataFrame(books_data)
df.to_csv("books.csv", index=False)
print("Data saved to books.csv")

# Display Data
print(df)

Data saved to books.csv
                                                Title    Price Availability
0                                A Light in the Attic  Â£51.77     In stock
1                                  Tipping the Velvet  Â£53.74     In stock
2                                          Soumission  Â£50.10     In stock
3                                       Sharp Objects  Â£47.82     In stock
4               Sapiens: A Brief History of Humankind  Â£54.23     In stock
5                                     The Requiem Red  Â£22.65     In stock
6   The Dirty Little Secrets of Getting Your Dream...  Â£33.34     In stock
7   The Coming Woman: A Novel Based on the Life of...  Â£17.93     In stock
8   The Boys in the Boat: Nine Americans and Their...  Â£22.60     In stock
9                                     The Black Maria  Â£52.15     In stock
10     Starving Hearts (Triangular Trade Trilogy, #1)  Â£13.99     In stock
11                              Shakespeare's Sonnets  Â£20.66  

---